尝试展示一下现有数据有什么特点

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df_exports = pd.read_excel(r'..\..\data\raw\Q3\前三问出口数据整理版.xlsx')
df_exports.head()

# delete the last 
df_exports = df_exports.drop('2025',axis=1).melt(id_vars=['Country'],var_name='Year',value_name='Exports')
df_exports.head()

# plot the data

# df_imports = pd.read_excel(r'..\..\data\raw\Q3\前三问进口数据整理版.xlsx')
# df_imports.head()



,Country,Year,Exports
0,Afghanistan,2020,215724
1,Albania,2020,46717
2,Algeria,2020,50300426
3,Angola,2020,0
4,Anguilla,2020,3480


使用冲击评估的方法来研究关税的影响:
1. 时间序列来预测无关税条件下的进口增长
2. 通过历史数据构建弹性函数
3. 明确弹性函数对时间序列分析下的进口增长的影响